In [1]:
import pandas as pd
import re

In [2]:
instructions = {
    'ADD': '18',
    'AND': '40',
    'COMP': '28',
    'DIV': '24',
    'J': '3C',
    'JEQ': '30',
    'JGT': '34',
    'JLT': '38',
    'JSUB': '48',
    'LDA': '00',
    'LDCH': '50',
    'LDL': '08',
    'LDX': '04',
    'MUL': '20',
    'OR': '44',
    'RD': 'D8',
    'RSUB': '4C',
    'STA': '0C',
    'STCH': '54',
    'STL': '14',
    'STSW': 'E8',
    'STX': '10',
    'SUB': '1C',
    'TD': 'E0',
    'TIX': '2C',
    'WD': 'DC',
    'FIX': ['C4', 1],
    'FLOAT': ['CO', 1],
    'HIO': ['F4', 1],
    'NORM': ['C8', 1],
    'SIO': ['F0', 1],
    'TIO': ['F8', 1],
}


In [3]:
def open_file(file):
    with open(file) as f:
        program = [re.sub(r'\s+', ' ', line).strip().upper() for line in f]
    return program

In [4]:
fout = open("zeee.txt", "w", encoding='utf-8')
for ind in df.index:
    if(df.Label[ind] == ' '):
        fout.write('\t\t{0}\t{1}\n'.format(df.Mnemonic[ind].ljust(8, ' '), df.Value[ind]).ljust(8, ' '))
        
    else:
        fout.write('{0}\t{1}\t{2}\n'.format(df.Label[ind].ljust(8, ' '), df.Mnemonic[ind].ljust(8, ' '), df.Value[ind].ljust(8, ' ')))
fout.close()

NameError: name 'df' is not defined

In [ ]:
def prog_name(df):
    return df.iloc[0].Label.ljust(6, 'x')

In [5]:
def return_df(program):
    dict = []
    for line in program:
        
        temp = line.split(" ")[1: 4]
        if len(temp) < 1:
            continue
        if temp[0] == '.':
            continue
        if temp[0] in instructions:
            temp = temp[: 2] 
        
        if temp[0]  == 'RSUB':
            label = ' '
            mnemonic = temp[0]
            value = ' '
            
        elif temp[0]  == 'END':
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif temp[1] == 'RSUB':
            label = temp[0]
            mnemonic = temp[1]
            value = ' '
        
            
        elif temp[0] in instructions:
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif len(temp) >= 3:
            label = temp[0]
            mnemonic = temp[1]
            value = temp[2]
        

        dict.append({
            'Label':label,
            'Mnemonic': mnemonic,
            'Value': value
        })
    df = pd.DataFrame(dict)

    if df.iloc[df.index.stop -1].Mnemonic != 'END':
        df2 = pd.DataFrame([[' ', 'END', df.iloc[0].Value]], columns=df.columns)
        df = pd.concat([df, df2], ignore_index = True)
    return df

In [6]:
program = open_file('in.txt')
df = return_df(program)
df

,Label,Mnemonic,Value
0,COPY,START,1000
1,FIRST,STL,RETADR
2,CLOOP,JSUB,RDREC
3,,LDA,LENGTH
4,,COMP,ZERO
5,,JEQ,ENDFIL
6,,JSUB,WRREC
7,,J,CLOOP
8,ENDFIL,LDA,EOF
9,,STA,BUFFER


In [7]:
def location_counter(df):
    start_index = df.Value[0]
    list_counter = [" ", start_index]
    counter = start_index
    for i in range(1, df.index.stop):

        if df.Mnemonic[i] in instructions:
            if type(instructions[df.Mnemonic[i]]) == list:
                temp = hex(int(counter, 16) + 1)
            else:
                temp = hex(int(counter, 16) + 3)
            
        if df.Mnemonic[i] == 'WORD':
            temp = hex(int(counter, 16) + 3)
        if df.Mnemonic[i] == 'BYTE':
            value = df.Value[i].split('\'')
            length = len(value[1])
            if value[0] == 'C':
                temp = hex(int(counter, 16) + int(length))
            else:
                temp = hex(int(counter, 16) + int(int(length)/2))
                
        if df.Mnemonic[i] == 'RESW':
            temp = hex(int(counter, 16) + int(df.Value[i]) *3)
            
        if df.Mnemonic[i] == 'RESB':
            temp = hex(int(counter, 16) + int(df.Value[i]))
            
    
        counter = temp.split('x')[1].rjust(4, '0').upper()
        list_counter.append(counter)
        
    df.insert(0, 'Location_Counter', list_counter[:-1])
    return df

In [8]:
location_counter(df)

,Location_Counter,Label,Mnemonic,Value
0,,COPY,START,1000
1,1000,FIRST,STL,RETADR
2,1003,CLOOP,JSUB,RDREC
3,1006,,LDA,LENGTH
4,1009,,COMP,ZERO
5,100C,,JEQ,ENDFIL
6,100F,,JSUB,WRREC
7,1012,,J,CLOOP
8,1015,ENDFIL,LDA,EOF
9,1018,,STA,BUFFER


In [10]:
def symbol_table(df):
    list = []
    for ind in range(1, df.index.stop):
        if df.Label[ind] != ' ':
            list.append({df.Label[ind]:df.Location_Counter[ind]})
    return list
list = symbol_table(df)

In [32]:
for dic in list:
    for key, value in dic.items():
        print(key, value)

FIRST 1000
CLOOP 1003
ENDFIL 1015
EOF 102A
THREE 102D
ZERO 1030
RETADR 1033
LENGTH 1036
BUFFER 1039
RDREC 2039
RLOOP 203F
EXIT 2057
INPUT 205D
MAXLEN 205E
TEST 2061


In [34]:
for dic in list:
    for key, value in dic.items():
        print('{0}\t{1}\n'.format(key.ljust(8, ' '), value))

FIRST   	1000

CLOOP   	1003

ENDFIL  	1015

EOF     	102A

THREE   	102D

ZERO    	1030

RETADR  	1033

LENGTH  	1036

BUFFER  	1039

RDREC   	2039

RLOOP   	203F

EXIT    	2057

INPUT   	205D

MAXLEN  	205E

TEST    	2061

